In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [64]:
from cleaner import clean_preliminary,to_categorical_for_cols,remove_nominal_cols,clean_sim_filled_data
from predictor import train_fill_ml_na,fill_ml_na_col,fill_ml_na
import pandas as pd
import math
import os
import utils
import constants as const
import numpy as np
import pickle

from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

## Reading and Preparing Train Data

The series of functions applied to the training data prepare the data as described in Approach 2 in our report. We briefly describe the use case of each funtion below: <br> 
- *clean_preliminary()*:
    - Handles each columns individually
    - Drops the unwanted rows and columns
- *to_categorical_for_cols()*:
    - Encodes *fuel_type* using dummy variables and *category* using MultiLabelBinarizer()
- *remove_nominal_cols()*:
    - Removes columns that are not needed for further analysis.

In [93]:
df = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [ ]:
df = clean_preliminary(df)
df = to_categorical_for_cols(df)
df = remove_nominal_cols(df)

test = clean_preliminary(test,is_test=True)
test = to_categorical_for_cols(test)
test = remove_nominal_cols(test)

### Filling Null Values
We have two ways to fill the null values. The training of both these processes take time, further, the saved ML models are very heavy to be pushed on the GitHub repository. We therefore save the data after applying the necessary approach.

#### Machine Learning Based Approach
The function *fill_ml_na()* is used to train Machine learning models, to predict the missing values for each column. In practice, we use RandomizedSearchCV with 200 iterations and 5-fold cross validation. <br>
Since the trained models are heavy, we do not include them in the repository, you can run the below code cell where we have set the number of iterations as 1 and k as 2 for cross validation to allow a quick sample.

In [81]:
# filled_train = fill_ml_na(df,training=True,num_iter=1,k_splits=2)
filled_train = fill_ml_na(df)
filled_test = fill_ml_na(test)

/Users/kanav7/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.24.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/kanav7/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.24.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/kanav7/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.24.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/kanav7/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.24.2 when using vers

make_model has not been trained
omv has not been trained


/Users/kanav7/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.24.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


Alternatively, you can use the below code cell to access the training data we have saved after filling null values using the ML models.

In [77]:
filled_train = pd.read_csv('data/Filled_df/train_ML_filled.csv')
filled_test = pd.read_csv('data/Filled_df/test_ML_filled.csv')

#### Similarity Based Approach


In [83]:
##TODO

Alternatively, you can use the below code cell to access the training data we have saved after filling null values using the ML models.

In [58]:
filled_train = pd.read_csv('data/Filled_df/train_sim_filled.csv')
filled_test = pd.read_csv('data/Filled_df/test_sim_filled.csv')

filled_train = clean_sim_filled_data(filled_train)
filled_test = clean_sim_filled_data(filled_test,is_test=True)

## Train LBGM

In [84]:
X = filled_train.drop(['index','price'],axis=1)
Y = filled_train.price

In [85]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [86]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=42)

In [87]:
lgb_model = lgb.LGBMRegressor(
    boosting_type="gbdt",
    num_iterations = 2500,
    learning_rate = 0.05,
    num_leaves=15,
    tree_learner='feature',
    max_depth =10,
    min_data_in_leaf=7,
    bagging_fraction = 1,
    bagging_freq = 100,
    reg_sqrt='True',
    metric ='rmse',
    feature_fraction = 0.6,
    random_state=42)

lgb_model.fit(X_train,y_train) 


preds_lgb_model = lgb_model.predict(X_test)
rmse_lgb = np.sqrt(mean_squared_error(y_test, preds_lgb_model))
print(" RMSE: %f" % (rmse_lgb ))

/Users/kanav7/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[LightGBM] [Warning] bagging_freq is set=100, subsample_freq=0 will be ignored. Current value: bagging_freq=100
 RMSE: 26831.449955


### Test Data

In [88]:
filled_test['Predicted'] = lgb_model.predict(filled_test.drop(['index'],axis=1))

In [89]:
filled_test['id'] = filled_test.index

In [90]:
filled_test.Predicted = filled_test.Predicted.apply(lambda x:round(x / 100.0) * 100.0)

In [91]:
submission = filled_test[['id','Predicted']]

In [53]:
submission.to_csv('data/submission.csv',index=False)